* ! Материалы взяты из курса "Интерактивный трёнажер по SQL". link: https://stepik.org/course/63054/syllabus

## Таблица "Нарушения ПДД", запросы корректировки"

**Задание**

Создать таблицу _fine_ следующей структуры:

| Поле | Описание |
| --- | --- |
| fine_id | ключевой столбец целого типа с автоматическим увеличением значения ключа на 1 |
| name | строка длиной 30 |
| number_plate | строка длиной 6 |
| violation | строка длиной 50 | 
| sum_fine | вещественное число, максимальная длина 8, количество знаков после запятой 2 |
| date_violation | дата |
| date_payment | дата |

In [ ]:
CREATE TABLE fine(fine_id INT PRIMARY KEY AUTO_INCREMENT,  
                  name VARCHAR(30),  
                  number_plate VARCHAR(6),  
                  violation VARCHAR(50),
                  sum_fine DECIMAL(8,2),
                  date_violation DATE,
                  date_payment DATE);

**Задание**  
В таблицу уже внесены 5 записей. Добавить в таблицу записи с ключевыми значениями:

| fine_id | name  | number_plate | violation | sum_fine | date_violation | date_payment |
| --- | ---  | --- | --- | --- | --- | --- |
| 6 | Баранов П.Е. | Р523ВТ | Превышение скорости (от 40 до 60) | | 2020-02-14 | |
| 7 | Абрамова К.А.| О111АВ | Проезд на запрещающий сигнал | | 2020-02-23 | |
| 8 | Яковлев Г.Р. | Т330ТТ	| Проезд на запрещающий сигнал | | 2020-03-03 | |


In [ ]:
INSERT INTO fine(name, number_plate, violation, date_violation)
VALUES
       ("Баранов П.Е.", "Р523ВТ", "Превышение скорости(от 40 до 60)", "2020-02-14"),
       ("Абрамова К.А.", "О111АВ", "Проезд на запрещающий сигнал", "2020-02-23"), 
       ("Яковлев Г.Р.", "Т330ТТ", "Проезд на запрещающий сигнал", "2020-03-03");

**Задание**

Занести в таблицу _fine_ суммы штрафов, которые должен оплатить водитель, в соответствии с данными из таблицы _traffic_violation_. При этом суммы заносить только в пустые поля столбца _sum_fine_.  
Таблица _traffic_violation_ создана и заполнена.  
Важно! Сравнение значения столбца с пустым значением осуществляется с помощью оператора _IS NULL_.

In [ ]:
UPDATE fine f, traffic_violation tv
SET f.sum_fine = tv.sum_fine
WHERE f.sum_fine IS NULL AND f.violation = tv.violation;

SELECT * FROM fine;

**Задание**  

Вывести фамилию, номер машины и нарушение только для тех водителей, которые на одной машине нарушили одно и то же правило   два и более раз. При этом учитывать все нарушения, независимо от того оплачены они или нет. Информацию отсортировать в алфавитном порядке, сначала по фамилии водителя, потом по номеру машины и, наконец, по нарушению.

In [ ]:
SELECT name, number_plate, violation
FROM fine
GROUP BY name, number_plate, violation
HAVING COUNT(*) > 1
ORDER BY name, number_plate, violation

**Задание**  

В таблице _fine_ увеличить в два раза сумму неоплаченных штрафов для отобранных на предыдущем шаге записей. 

In [ ]:
UPDATE fine f,
           (SELECT name,
                   number_plate,
                   violation
                   FROM fine
                   GROUP BY name, number_plate, violation
                   HAVING COUNT(*) > 1) query_in
SET f.sum_fine = f.sum_fine * 2
WHERE f.date_payment IS NULL AND f.name = query_in.name AND
                               f.number_plate = query_in.number_plate AND
                               f.violation = query_in.violation;

SELECT * FROM fine;

**Задание**  

Водители оплачивают свои штрафы. В таблице payment занесены даты их оплаты:

| payment_id | name | number_plate | violation | date_violation | date_payment |
| --- | --- | --- |  --- | ---| ---| 
| 1 | Яковлев Г.Р. | М701АА | Превышение скорости (от 20 до 40) | 2020-01-12 | 2020-01-22|
| 2 | Баранов П.Е. | Р523ВТ | Превышение скорости (от 40 до 60) | 2020-02-14 | 2020-03-06|
| 3 | Яковлев Г.Р. | Т330ТТ | Проезд на запрещающий сигнал | 2020-03-03 | 2020-03-23 |

Необходимо:

- в таблицу _fine_ занести дату оплаты соответствующего штрафа из таблицы _payment_ ; 
- уменьшить начисленный штраф в таблице _fine_ в два раза  (только для тех штрафов, информация о которых занесена в таблицу payment) , если оплата произведена не позднее 20 дней со дня нарушения.

In [ ]:
UPDATE fine f, payment p
SET f.date_payment = p.date_payment,
    f.sum_fine = IF(DATEDIFF(p.date_payment, f.date_violation) > 20, f.sum_fine, f.sum_fine/2)
WHERE f.date_payment IS NULL 
      AND f.name = p.name
      AND f.number_plate = p.number_plate
      AND f.violation = p.violation
      AND f.date_violation = p.date_violation;

SELECT * FROM fine;

**Задание**  

Создать новую таблицу _back_payment_, куда внести информацию о неоплаченных штрафах (Фамилию и инициалы водителя, номер машины, нарушение, сумму штрафа  и  дату нарушения) из таблицы _fine_.

In [ ]:
CREATE TABLE back_payment
SELECT name, number_plate, violation, sum_fine, date_violation
FROM fine
WHERE date_payment IS NULL;

SELECT * FROM back_payment;

## База данных «Интернет-магазин книг», запросы на выборку
**Логическая модель базы данных**
![jupyter](https://ucarecdn.com/bad26356-5e34-4945-a9d4-0748686a6b54/)

**Задание**

Вывести информацию о каждом заказе: его номер, кто его сформировал (фамилия пользователя) и его стоимость (сумма произведений количества заказанных книг и их цены), в отсортированном по номеру заказа виде. Последний столбец назвать _Стоимость_.

In [ ]:
SELECT buy_id, name_client, SUM(book.price*buy_book.amount) AS Стоимость
FROM buy LEFT JOIN client USING(client_id) LEFT JOIN buy_book USING(buy_id) INNER JOIN book USING(book_id)
GROUP BY buy_id
ORDER BY 1

**Задание**

Вывести номера заказов (*buy_id*) и названия этапов, на которых они в данный момент находятся. Если заказ доставлен –  информацию о нем не выводить. Информацию отсортировать по возрастанию *buy_id*.

In [ ]:
SELECT buy_id, name_step
FROM buy LEFT JOIN buy_step USING(buy_id) INNER JOIN step USING(step_id)
WHERE date_step_beg IS NOT Null AND date_step_end IS Null
ORDER BY 1

**Задание**

В таблице *city* для каждого города указано количество дней, за которые заказ может быть доставлен в этот город (рассматривается только этап Транспортировка). Для тех заказов, которые прошли этап транспортировки, вывести количество дней за которое заказ реально доставлен в город. А также, если заказ доставлен с опозданием, указать количество дней задержки, в противном случае вывести 0. В результат включить номер заказа (*buy_id*), а также вычисляемые столбцы *Количество_дней* и *Опоздание*. Информацию вывести в отсортированном по номеру заказа виде.

In [ ]:
SELECT buy_id,
       DATEDIFF(date_step_end, date_step_beg) AS Количество_дней,
       GREATEST(DATEDIFF(date_step_end, date_step_beg) - days_delivery, 0) AS Опоздание
FROM buy_step JOIN buy USING(buy_id)
              JOIN client USING(client_id)
              JOIN city USING(city_id)
WHERE step_id = 3 AND date_step_end IS NOT Null

**Задание**

Выбрать всех клиентов, которые заказывали книги Достоевского, информацию вывести в отсортированном по алфавиту виде. В решении используйте фамилию автора, а не его *id*.

In [ ]:
SELECT DISTINCT name_client
FROM client JOIN buy USING(client_id)
    WHERE buy_id = ANY(
        SELECT buy_id FROM buy_book WHERE book_id = ANY(
            SELECT book_id FROM book JOIN author USING(author_id) WHERE name_author LIKE "%Достоевский%"))
ORDER BY 1

**Задание**

Вывести жанр (или жанры), в котором было заказано больше всего экземпляров книг, указать это количество . Последний столбец назвать *Количество*.

In [ ]:
SELECT name_genre, SUM(buy_book.amount) AS Количество
FROM genre LEFT JOIN book USING(genre_id) JOIN buy_book USING(book_id)
GROUP BY name_genre
HAVING SUM(buy_book.amount) = 
    (SELECT SUM(buy_book.amount)
     FROM genre LEFT JOIN book USING(genre_id) JOIN buy_book USING(book_id)
     GROUP BY name_genre
     ORDER BY 1 DESC
     LIMIT 1)

**Задание**

Сравнить ежемесячную выручку от продажи книг за текущий и предыдущий годы. Для этого вывести год, месяц, сумму выручки в отсортированном сначала по возрастанию месяцев, затем по возрастанию лет виде. Название столбцов: *Год*, *Месяц*, *Сумма*.

In [ ]:
SELECT YEAR(date_step_end) AS "Год",
       MONTHNAME(date_step_end) AS "Месяц",
       SUM(buy_book.amount * book.price) AS "Сумма"
FROM buy_book JOIN buy_step USING(buy_id) JOIN book USING(book_id)
WHERE step_id = 1 AND date_step_end IS NOT NULL
GROUP BY 1, 2
UNION
SELECT YEAR(date_payment) AS "Год",
       MONTHNAME(date_payment) AS "Месяц",
       SUM(amount * price) AS "Сумма"
FROM buy_archive 
GROUP BY 1, 2
ORDER BY 2, 1

**Задание**

Для каждой отдельной книги необходимо вывести информацию о количестве проданных экземпляров и их стоимости за 2020 и 2019 год . Вычисляемые столбцы назвать *Количество* и *Сумма*. Информацию отсортировать по убыванию стоимости.

In [ ]:
SELECT title, SUM(amount) AS Количество, SUM(price) AS Сумма
FROM(
    SELECT title, SUM(bb.amount) AS amount, SUM(bb.amount * b.price) AS price
    FROM buy_book bb JOIN buy_step bs USING(buy_id) JOIN book b USING(book_id)
    WHERE step_id = 1 AND date_step_end IS NOT NULL
    GROUP BY 1
    UNION
    SELECT title, SUM(ba.amount), SUM(ba.amount * ba.price) AS price
    FROM buy_archive ba JOIN book USING(book_id)
    GROUP BY 1
) t1
GROUP BY 1
ORDER BY 3 DESC

## База данных «Интернет-магазин книг», запросы корректировки
**Задание**  

В таблицу *buy_book* добавить заказ с номером 5. Этот заказ должен содержать книгу Пастернака «Лирика» в количестве двух экземпляров и книгу Булгакова «Белая гвардия» в одном экземпляре.

In [ ]:
INSERT INTO buy_book(book_id, buy_id, amount)
VALUES
   ((SELECT book_id FROM book WHERE author_id =
    (SELECT author_id FROM author WHERE name_author LIKE "%Пастернак%")
    AND title="Лирика"), 5, 2),
   ((SELECT book_id FROM book WHERE author_id =
    (SELECT author_id FROM author WHERE name_author LIKE "%Булгаков%")
    AND title="Белая гвардия"), 5, 1);

SELECT * FROM buy_book

**Задание**  

Количество тех книг на складе, которые были включены в заказ с номером 5, уменьшить на то количество, которое в заказе с номером 5  указано.

In [ ]:
UPDATE book, buy_book
SET book.amount = book.amount - buy_book.amount
WHERE buy_book.buy_id=5 AND book.book_id = buy_book.book_id;

SELECT * FROM book

**Задание**  

Создать счет (таблицу *buy_pay*) на оплату заказа с номером 5, в который включить название книг, их автора, цену, количество заказанных книг и  стоимость. Последний столбец назвать *Стоимость*. Информацию в таблицу занести в отсортированном по названиям книг виде.

In [ ]:
CREATE TABLE buy_pay AS
SELECT book.title, author.name_author, book.price, buy_book.amount, buy_book.amount * book.price AS "Стоимость"
FROM book, buy_book, author
WHERE buy_book.buy_id = 5 AND buy_book.book_id = book.book_id AND book.author_id = author.author_id
ORDER BY 1;

SELECT * FROM buy_pay

**Задание**  

Создать общий счет (таблицу *buy_pay*) на оплату заказа с номером 5. Куда включить номер заказа, количество книг в заказе (название столбца *Количество*) и его общую стоимость (название столбца *Итого*).

In [ ]:
CREATE TABLE buy_pay AS
SELECT buy_book.buy_id, SUM(buy_book.amount) AS "Количество", SUM(buy_book.amount * book.price) AS "Итого"
FROM book, buy_book, author
WHERE buy_book.buy_id = 5 AND buy_book.book_id = book.book_id AND book.author_id = author.author_id
GROUP BY 1;

SELECT * FROM buy_pay

**Задание**  

В таблицу *buy_step* для заказа с номером 5 включить все этапы из таблицы *step*, которые должен пройти этот заказ. В столбцы *date_step_beg* и *date_step_end* всех записей занести *Null*.

In [ ]:
INSERT INTO buy_step(buy_id, step_id)
(SELECT buy_id, step_id FROM buy CROSS JOIN step WHERE buy.buy_id = 5);

SELECT * FROM buy_step

**Задание**  

В таблицу *buy_step* занести дату 12.04.2020 выставления счета на оплату заказа с номером 5.

Правильнее было бы занести не конкретную, а текущую дату. Это можно сделать с помощью функции *Now()*. Но при этом в разные дни будут вставляться разная дата, и задание нельзя будет проверить, поэтому  вставим дату 12.04.2020.

In [ ]:
UPDATE buy_step
SET date_step_beg = "2020-04-12"
WHERE buy_id = 5 AND step_id = 1;

SELECT * FROM buy_step

**Задание**  

Завершить этап **«Оплата»** для заказа с номером 5, вставив в столбец *date_step_end* дату 13.04.2020, и начать следующий этап (**«Упаковка»**), задав в столбце *date_step_beg* для этого этапа ту же дату.

Реализовать два запроса для завершения этапа и начала следующего. Они должны быть записаны в общем виде, чтобы его можно было применять для любых этапов, изменив только текущий этап. Для примера пусть это будет этап **«Оплата»**.

In [ ]:
SET @buy_id = 5;
SET @date = "2020-04-13";
SET @step_id = 1;

UPDATE buy_step
SET date_step_end = @date
WHERE buy_id = @buy_id AND step_id = @step_id;

UPDATE buy_step
SET date_step_beg = @date
WHERE buy_id = @buy_id AND step_id = @step_id + 1;

SELECT * FROM buy_step